In [2]:
import tensorflow as tf
#import tensorflow.compat.v1 as tf
import os
import _pickle as cPickle
import numpy as np
#tf.compat.v1.disable_eager_execution()
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() 
dir="./cifar-10-batches-py/"
print (os.listdir(dir))

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [3]:
def load_data(filename):
    '''read data from datafile'''
    with open(filename,'rb') as f:
        data=cPickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data=[]
        all_labels=[]
        for filename in filenames:
            data,labels=load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
            
            self._data=np.vstack(all_data)
            self._data=self._data/127.5-1
            self._labels=np.hstack(all_labels)
            #debug
            print(self._data.shape)
            print(self._labels.shape)
            self._num_examples=self._data.shape[0]
            self._need_shuffle=need_shuffle
            self._indicator=0
            if self._need_shuffle:
                self._shuffle_data()
            
    def _shuffle_data(self):
        #[0,1,2,3,4,5]->[5,3,2,4,0,1]
        p=np.random.permutation(self._num_examples)
        self._data=self._data[p]
        self._labels=self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size examples as a batch."""
        end_indicator=self._indicator+batch_size
        if(end_indicator>self._num_examples):
            if(self._need_shuffle):
                self._shuffle_data()
                self._indicator=0
                end_indicator=batch_size
            else:
                raise Exception("have no more example")
        if(end_indicator>self._num_examples):
            raise Exception("batch size is large than all example")
        batch_data=self._data[self._indicator:end_indicator]
        batch_labels=self._labels[self._indicator:end_indicator]
        self._indicator=end_indicator
        return batch_data,batch_labels
train_filenames=[os.path.join(dir,'data_batch_%d' % i)for i in range(1,6)]
test_filenames=[os.path.join(dir,'test_batch')]
train_data=CifarData(train_filenames,True)
test_data=CifarData(test_filenames,False)
#batch_data,batch_labels=train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)

(10000, 3072)
(10000,)
(20000, 3072)
(20000,)
(30000, 3072)
(30000,)
(40000, 3072)
(40000,)
(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [8]:
tf.reset_default_graph()
x=tf.placeholder(tf.float32,[None,3072])
#None,eg:[0,5,6,3]
y=tf.placeholder(tf.int64,[None])

hidden1=tf.layers.dense(x,100,activation=tf.nn.relu)
hidden2=tf.layers.dense(hidden1,100,activation=tf.nn.relu)
hidden3=tf.layers.dense(hidden2,50,activation=tf.nn.relu)
y_=tf.layers.dense(hidden3,10)

#mean square loss
'''
#course:1+e^x
#api:e^x/sum(e^x)
#[[0.01,0,02,0,8...,0.03],[]]
p_y=tf.nn.softmax(y_)
#5->[0,0,0,0,0,1,0,0,0,0]
y_one_hot=tf.one_hot(y,10,dtype=tf.float32)
loss=tf.reduce_mean(tf.square(y_one_hot-p_y))
'''

loss=tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#y_ -> sofmax
#y -> onehot
#loss = ylogy_ `

#index
predict=tf.argmax(y_,1)
#[1,0,1,0,0,1]
correct_prediction=tf.equal(predict,y)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float64))
with tf.name_scope('train_op'):
    train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)

In [9]:
init=tf.compat.v1.global_variables_initializer()
batch_size=20
train_steps=10000

'''
g = tf.Graph()
with g.as_default():
    hello = tf.constant('hello')
    sess = tf.compat.v1.Session(graph=g)
'''
with tf.compat.v1.Session() as sess:
    a=sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels=train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run(
            [loss,accuracy,train_op],
            feed_dict={
                x:batch_data,y:batch_labels
            })
        if (i%500==0):
            print('[Train]Step:%d,loss:%4.5f,acc:%4.5f'\
                  %(i,loss_val,acc_val))
        if (i%5000==0):
            test_batch_data,test_batch_labels=test_data.next_batch(batch_size)
            test_acc_val=sess.run([accuracy],feed_dict={x:test_batch_data,y:test_batch_labels})

[Train]Step:0,loss:2.30134,acc:0.05000
[Train]Step:500,loss:1.90306,acc:0.35000
[Train]Step:1000,loss:1.59015,acc:0.30000
[Train]Step:1500,loss:1.62176,acc:0.45000
[Train]Step:2000,loss:1.78897,acc:0.50000
[Train]Step:2500,loss:1.64806,acc:0.45000
[Train]Step:3000,loss:1.63098,acc:0.35000
[Train]Step:3500,loss:1.76306,acc:0.30000
[Train]Step:4000,loss:1.86501,acc:0.35000
[Train]Step:4500,loss:1.41276,acc:0.50000
[Train]Step:5000,loss:1.22714,acc:0.55000
[Train]Step:5500,loss:0.84604,acc:0.65000
[Train]Step:6000,loss:1.29641,acc:0.55000
[Train]Step:6500,loss:1.35874,acc:0.65000
[Train]Step:7000,loss:1.77625,acc:0.45000
[Train]Step:7500,loss:1.49431,acc:0.50000
[Train]Step:8000,loss:1.31618,acc:0.55000
[Train]Step:8500,loss:1.27494,acc:0.45000
[Train]Step:9000,loss:1.08173,acc:0.65000
[Train]Step:9500,loss:1.39515,acc:0.50000
